## FairFace In!

In [ ]:
import kagglehub
import os
import random

directory_real = kagglehub.dataset_download("aibloy/fairface")
path_real = os.path.join(directory_real, "FairFace")

print("Path to dataset files:", directory_real)


100%|██████████| 550M/550M [00:09<00:00, 63.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/aibloy/fairface/versions/1


In [ ]:
import pandas as pd

image_path = "/root/.cache/kagglehub/datasets/aibloy/fairface/versions/1/FairFace"

train_fair = pd.DataFrame(pd.read_csv(f'{path_real}/train_labels.csv'))
train_fair['image_id'] = train_fair['file'].astype(str).apply(lambda x: x.split('/')[-1])
train_fair["gender"] = train_fair["gender"].replace({"Male": 1, "Female": 0}).astype(int)
train_fair["race"] = train_fair["race"].apply(lambda x: 1 if x == "White" else 0)
train_fair['label'] = 1
train_fair = train_fair[['image_id', 'file', 'age', 'race', 'gender', 'label']]
train_fair = train_fair.rename(columns={'file': 'image_path', 'race': 'ethnicity'})

train_fair.loc[:, 'image_path'] = train_fair['image_id'].apply(lambda x: os.path.join(image_path, "train", x))

val_fair = pd.DataFrame(pd.read_csv(f'{path_real}/val_labels.csv'))
val_fair['image_id'] = val_fair['file'].astype(str).apply(lambda x: x.split('/')[-1])
val_fair["gender"] = val_fair["gender"].replace({"Male": 1, "Female": 0}).astype(int)
val_fair["race"] = val_fair["race"].apply(lambda x: 1 if x == "White" else 0)
val_fair['label'] = 1
val_fair = val_fair[['image_id', 'file', 'age', 'race', 'gender', 'label']]
val_fair = val_fair.rename(columns={'file': 'image_path', 'race': 'ethnicity'})

val_fair.loc[:, 'image_path'] = val_fair['image_id'].apply(lambda x: os.path.join(image_path, "val", x))


<ipython-input-2-be5d6d346fb4>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_fair["gender"] = train_fair["gender"].replace({"Male": 1, "Female": 0}).astype(int)
<ipython-input-2-be5d6d346fb4>:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  val_fair["gender"] = val_fair["gender"].replace({"Male": 1, "Female": 0}).astype(int)


In [ ]:
fair_face = pd.concat([train_fair, val_fair])

white = fair_face.where(fair_face['ethnicity'] == 1).dropna()
non_white = fair_face.where(fair_face['ethnicity'] == 0).dropna()

non_white_sample = non_white.sample(n=white.shape[0], random_state=42)

balanced_fair_face = pd.concat([white, non_white_sample])
balanced_fair_face[['gender', 'ethnicity', 'label']] = balanced_fair_face[['gender', 'ethnicity', 'label']].astype(int)

balanced_fair_face

,image_id,image_path,age,ethnicity,gender,label
5,6.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,20-29,1,1,1
8,9.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,10-19,1,1,1
15,16.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,30-39,1,0,1
23,24.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,30-39,1,0,1
24,25.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,20-29,1,0,1
...,...,...,...,...,...,...
5474,5475.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,30-39,0,1,1
76036,76037.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,20-29,0,1,1
54603,54604.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,0-2,0,1,1
72168,72169.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,40-49,0,1,1


In [ ]:
# hist = balanced_fair_face['age'].hist(bins=3)
balanced_fair_face['age'].value_counts().sort_index()

,count
age,
0-2,822
10-19,3459
20-29,11180
3-9,3947
30-39,8588
40-49,4686
50-59,2862
60-69,1313
more than 70,367


In [ ]:
balanced_fair_face.to_csv('fair_face_balanced.csv', index=False)


## Fingerprints In!

In [ ]:
!git clone https://github.com/ningyu1991/ArtificialGANFingerprints.git
%cd ArtificialGANFingerprints
!pip install -r requirements.txt


In [ ]:
import shutil
import os
import kagglehub
import pandas as pd

directory_real = kagglehub.dataset_download("aibloy/fairface")
path_real = os.path.join(directory_real, "FairFace")

dir = "/root/.cache/kagglehub/datasets/aibloy/fairface/versions/1/FairFace"

df = pd.read_csv("/content/fair_face_balanced.csv")

destination_folder = '/content/fair_face_selected_images/'

os.makedirs(destination_folder, exist_ok=True)

for img_path in df['image_path']:
    if os.path.exists(img_path):
        shutil.copy(img_path, destination_folder)
    else:
        print(f"File not found: {img_path}")

In [ ]:
import os
from PIL import Image

input_folder = '/content/fair_face_selected_images'
output_folder = '/content/fair_face_selected_images_resized'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)

    if filename.endswith(('.png', '.jpg', '.jpeg')):
        try:
            img = Image.open(file_path)

            img_resized = img.resize((128, 128))

            output_path = os.path.join(output_folder, filename)
            img_resized.save(output_path)
        except Exception as e:
            print(f'Error processing {filename}: {e}')


In [ ]:
# !python3 train.py \
# --data_dir /content/fair_face_selected_images_resized \
# --use_celeba_preprocessing \
# --image_resolution 128 \
# --output_dir /content/fair_face_inject \
# --fingerprint_length 100 \
# --batch_size 64

In [ ]:
!python embed_fingerprints.py --encoder_path /content/CelebA_128x128_encoder.pth --data_dir /content/fair_face_selected_images_resized --output_dir /content/fair_face_inject --image_resolution 128 --batch_size 64


Loading image folder /content/fair_face_selected_images_resized ...
Finished. Loading took 0.13s
Fingerprinting the images...
100% 557/557 [00:57<00:00,  9.68it/s]
^C


In [ ]:
df['fingerprint_path'] = df['image_id'].apply(
    lambda x: os.path.join('/content/fair_face_inject/fingerprinted_images', x)
)

In [ ]:
df.to_csv('/content/fc')